In [0]:
from pyspark import SparkFiles
url = "/Data_repository/main/Tweets.csv"
spark.sparkContext.addFile(url)
TweeterSent = spark.read.csv("file://"+SparkFiles.get("Tweets.csv"), header = True)

In [0]:
for col in TweeterSent.dtypes:
    print(col[0]+" , "+col[1])

tweet_id , string
airline_sentiment , string
airline_sentiment_confidence , string
negativereason , string
negativereason_confidence , string
airline , string
airline_sentiment_gold , string
name , string
negativereason_gold , string
retweet_count , string
text , string
tweet_coord , string
tweet_created , string
tweet_location , string
user_timezone , string


In [0]:
#Checking for null values in the df columns
from pyspark.sql.functions import col,isnan,when,count
isNullFram = TweeterSent.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in TweeterSent.columns])
display(isNullFram)

tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,155,68,5573,4229,179,14788,196,14805,205,214,13768,389,5010,5103


In [0]:
display(TweeterSent)

tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
570306133677760513,neutral,1.0,null,null,Virgin America,null,cairdin,null,0,@VirginAmerica What @dhepburn said.,null,2015-02-24 11:35:52 -0800,null,Eastern Time (US & Canada)
570301130888122368,positive,0.3486,null,0.0,Virgin America,null,jnardino,null,0,@VirginAmerica plus you've added commercials to the experience... tacky.,null,2015-02-24 11:15:59 -0800,null,Pacific Time (US & Canada)
570301083672813571,neutral,0.6837,null,null,Virgin America,null,yvonnalynn,null,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,null,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
570301031407624196,negative,1.0,Bad Flight,0.7033,Virgin America,null,jnardino,null,0,"""@VirginAmerica it's really aggressive to blast obnoxious """"entertainment"""" in your guests' faces & they have little recourse""",null,2015-02-24 11:15:36 -0800,null,Pacific Time (US & Canada)
570300817074462722,negative,1.0,Can't Tell,1.0,Virgin America,null,jnardino,null,0,@VirginAmerica and it's a really big bad thing about it,null,2015-02-24 11:14:45 -0800,null,Pacific Time (US & Canada)
570300767074181121,negative,1.0,Can't Tell,0.6842,Virgin America,null,jnardino,null,0,@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.,null,null,null,null
"it's really the only bad thing about flying VA""",null,2015-02-24 11:14:33 -0800,null,Pacific Time (US & Canada),null,null,null,null,null,null,null,null,null,null
570300616901320704,positive,0.6745,null,0.0,Virgin America,null,cjmcginnis,null,0,"@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)",null,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
570300248553349120,neutral,0.634,null,null,Virgin America,null,pilot,null,0,"@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP",null,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
570299953286942721,positive,0.6559,null,null,Virgin America,null,dhepburn,null,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",null,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)


In [0]:
#removing columns that have too many nulls (4000 and above )
empty_columns = ["negativereason","airline_sentiment_gold","negativereason_gold","tweet_coord","negativereason_confidence","tweet_location","user_timezone"]
for col in empty_columns:
    TweeterSent = TweeterSent.drop(col)

In [0]:
TweeterSent = TweeterSent.dropna()
TweeterSent.count()

Out[78]: 14448

In [0]:
# check if any df column contains null value
from pyspark.sql.functions import col,isnan,when,count
isNullFram = TweeterSent.select([count(when(
                            col(c).contains('NULL') | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in TweeterSent.columns])
display(isNullFram)

tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created
0,0,0,0,0,0,0,0


In [0]:
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer
from pyspark.ml import Pipeline
from pyspark.sql.functions import expr
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import SQLTransformer

# tokenize 
tokenizer = Tokenizer(inputCol="text", outputCol="vector")
#stop word 
StopWords = StopWordsRemover(inputCol="vector", outputCol="text_filtered")
# TF IDF 
ht = HashingTF(inputCol="text_filtered", outputCol="features")

# Encoding String Indexer 
SI_tweet_created = StringIndexer(inputCol='tweet_created',outputCol='tweet_created_Index')
SI_airline = StringIndexer(inputCol='airline',outputCol='airline_Index')
SI_name = StringIndexer(inputCol='name',outputCol='name_Index')
SI_airline_sentiment_confidence = StringIndexer(inputCol='airline_sentiment_confidence',outputCol='airline_sentiment_confidence_Index')
SI_retweet_count = StringIndexer(inputCol='retweet_count',outputCol='retweet_count_Index')
SI_airline_sentiment = StringIndexer(inputCol='airline_sentiment',outputCol='airline_sentiment_Index')

# Sql Transformer for selecting required columns
sqlDropCols = SQLTransformer(statement="SELECT airline_sentiment_Index, features FROM __THIS__")

# Logistic Regression
lr = LogisticRegression(labelCol = 'airline_sentiment_Index')

In [0]:
#Test and train split
train, test = TweeterSent.randomSplit([0.9, 0.1], seed=12345)
train.cache()
test.cache()

Out[81]: DataFrame[tweet_id: string, airline_sentiment: string, airline_sentiment_confidence: string, airline: string, name: string, retweet_count: string, text: string, tweet_created: string]

In [0]:
display(train)

tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created
567588278875213824,neutral,1.0,Delta,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to please passengers and Wall ... - Greenfield Daily Reporter http://t.co/LM3opxkxch,2015-02-16 23:36:05 -0800
567590027375702016,negative,1.0,Delta,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 #nothappy,2015-02-16 23:43:02 -0800
567591480085463040,negative,1.0,United,CPoutloud,0,@united yes. We waited in line for almost an hour to do so. Some passengers just left not wanting to wait past 1am.,2015-02-16 23:48:48 -0800
567592368451248130,negative,1.0,United,brenduch,0,"@united the we got into the gate at IAH on time and have given our seats and closed the flight. If you know people is arriving, have to wait",2015-02-16 23:52:20 -0800
567594449874587648,negative,1.0,Southwest,VahidESQ,0,"@SouthwestAir its cool that my bags take a bit longer, dont give me baggage blue balls-turn the carousel on, tell me it's coming, then not.",2015-02-17 00:00:36 -0800
567594579310825473,negative,1.0,United,brenduch,0,"@united and don't hope for me having a nicer flight some other time, try to do things right. You sold me those tickets with that connetion",2015-02-17 00:01:07 -0800
567595670463205376,negative,1.0,United,CRomerDome,0,@united I like delays less than you because I'm the one on the plane. Connect me with a voucher,2015-02-17 00:05:27 -0800
567614049425555457,negative,1.0,United,JustOGG,0,"@united, link to current status of flights/airports? Fly BWI-EWR-MCO this morning yet can't yet tell what any problems are except see snow.",2015-02-17 01:18:29 -0800
567617081336950784,negative,1.0,Southwest,mrshossruns,0,@SouthwestAir you guys there? Are we on hour 2 of our phone hold at 3am bc of volume or short staffing?,2015-02-17 01:30:32 -0800
567617486703853568,negative,1.0,United,feliciastoler,0,@united I tried 2 DM it would not go thru... not sure why,2015-02-17 01:32:09 -0800


In [0]:
display(test)

tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created
567680108002291712,positive,0.6645,Delta,TravellerLukose,0,@JetBlue No worries. Delay was minor and dealt with nicely. It was captain of flight 2324 by the way.,2015-02-17 05:40:59 -0800
567686758708817921,neutral,0.6890000000000001,Southwest,Tinygami,0,@SouthwestAir Is it a temporary site glitch or are you no longer offering flights from GRR to GEG after Feb? Can't find any online :(,2015-02-17 06:07:25 -0800
567702414157824000,negative,1.0,Southwest,lucasdavidson,0,@SouthwestAir on hold for 2 hours and then you hung up 3 Cancelled Flightled flights. Running out of daycare for our kids who are trapped at home.,2015-02-17 07:09:37 -0800
567715917983776772,negative,1.0,Southwest,sahandmirza,0,@SouthwestAir been on hold for >30 minutes about my Cancelled Flightled flight. Still not talked to anyone and flight is soon. What gives?,2015-02-17 08:03:17 -0800
567717415367090176,negative,1.0,Southwest,PorterHaney,0,@SouthwestAir the last 4 times I've arrived @LASairport our gate has been blocked by a slow to depart plane leading to 30-60 min delays,2015-02-17 08:09:14 -0800
567725676958998528,neutral,1.0,Delta,gigirey1,0,@JetBlue what's the link for #FlyFi,2015-02-17 08:42:03 -0800
567726138223771649,negative,1.0,US Airways,crshipferling,0,@USAirways your chairmans phone is down. what other number can i use?,2015-02-17 08:43:53 -0800
567726860638101504,negative,1.0,United,mlipschits,0,@united why am i mot getting my refund?,2015-02-17 08:46:46 -0800
567728285174743041,negative,1.0,US Airways,TheTaxDiva,0,@USAirways I did and it's been a disaster. You had me sitting on the runway only to bring the plane back to the gate smh,2015-02-17 08:52:25 -0800
567728481623359488,negative,1.0,US Airways,TheTaxDiva,0,@USAirways I expect something more than telling me to see an agent to rebook my flight...,2015-02-17 08:53:12 -0800


In [0]:
#parameter 
paramGrid = ParamGridBuilder() \
    .addGrid(ht.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "automatic", metricName = "accuracy")
evaluator.setLabelCol("airline_sentiment_Index")

Out[84]: MulticlassClassificationEvaluator_82f0255d2aa7

In [0]:
# creating pipeline
pipeline = Pipeline(stages = [SI_tweet_created,SI_airline,SI_name,SI_airline_sentiment_confidence,SI_retweet_count,SI_airline_sentiment,tokenizer, StopWords,ht, sqlDropCols, lr])

In [0]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator = evaluator,
                          numFolds=4)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train)

/databricks/spark/python/pyspark/ml/util.py:886: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
  warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)


In [0]:
# performing prediction on test data
prediction = cvModel.transform(test)

In [0]:
display(prediction)

airline_sentiment_Index,features,rawPrediction,probability,prediction
2.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(29, 273, 297, 314, 365, 639, 652, 699, 803, 903), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(0.803058857706326, 0.131575506149921, -0.9346343638562472))","Map(vectorType -> dense, length -> 3, values -> List(0.5928118534446123, 0.30289727387642296, 0.10429087267896467))",0.0
1.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(86, 105, 211, 330, 341, 433, 510, 593, 615, 654, 700, 807, 891), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(1.0033021948121297, 0.16790660976804264, -1.1712088045801723))","Map(vectorType -> dense, length -> 3, values -> List(0.646259262781835, 0.28028450421937057, 0.0734562329987944))",0.0
0.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(23, 63, 206, 330, 383, 389, 602, 640, 661, 726, 861, 892, 986, 999), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(2.158768490515224, -0.5799261732875833, -1.5788423172276402))","Map(vectorType -> dense, length -> 3, values -> List(0.9187244768580441, 0.05939984435314819, 0.02187567878880764))",0.0
0.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(29, 62, 221, 286, 288, 330, 389, 603, 635, 661, 666, 712, 999), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(2.7325731299376925, -1.9633380536871183, -0.7692350762505741))","Map(vectorType -> dense, length -> 3, values -> List(0.9622088881526311, 0.00878741323076015, 0.02900369861660876))",0.0
0.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(39, 61, 159, 189, 203, 305, 330, 369, 533, 536, 752, 873, 939, 989), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(1.1151339968757736, 0.2748525843094308, -1.389986581185204))","Map(vectorType -> dense, length -> 3, values -> List(0.6608272437965178, 0.2852058035242695, 0.05396695267921258))",0.0
1.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(66, 159, 297), values -> List(1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(0.0020514621435866118, 0.1252540246049415, -0.12730548674852826))","Map(vectorType -> dense, length -> 3, values -> List(0.33225088303304035, 0.37581346545910144, 0.2919356515078582))",1.0
0.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(25, 434, 519, 626, 726, 736), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(0.2503817566609905, 1.0600966956360314, -1.3104784522970223))","Map(vectorType -> dense, length -> 3, values -> List(0.2892495202058969, 0.6500209845522783, 0.060729495241824805))",1.0
0.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(20, 540, 560, 979), values -> List(1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(0.9440894229189081, 0.17976146262438558, -1.1238508855432938))","Map(vectorType -> dense, length -> 3, values -> List(0.6281041250163967, 0.2924746501013276, 0.07942122488227585))",0.0
0.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(39, 57, 61, 423, 429, 430, 482, 487, 736), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values -> List(1.8923135959541773, -1.0417448136618024, -0.8505687822923748))","Map(vectorType -> dense, length -> 3, values -> List(0.8948023241155936, 0.0475862768097567, 0.05761139907464981))",0.0
0.0,"Map(vectorType -> sparse, length -> 1000, indices -> List(16, 37, 330, 345, 354, 691, 736, 826), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> dense, length -> 3, values

In [0]:
# Instantiate metrics object
from pyspark.mllib.evaluation import MulticlassMetrics
predictionAndLabels = prediction.select(['prediction', 'airline_sentiment_Index']).withColumnRenamed('airline_sentiment_Index', 'label').rdd
metrics = MulticlassMetrics(predictionAndLabels)
metrics.confusionMatrix().toArray()

Out[89]: array([[835.,  52.,  25.],
       [165., 110.,  28.],
       [112.,  30.,  89.]])

In [0]:
from pyspark.sql.types import StructType, StructField, DoubleType, FloatType, StringType
# defining dataframe schema for metrics table inclusing productive metric terms as headers for each row of data labels
schema = StructType([
    StructField("Label", StringType(), True), 
    StructField("True Positive Rate", DoubleType(), True),
    StructField("False Positive Rate", DoubleType(), True),
    StructField("Precision", DoubleType(), True),
    StructField("Recall", DoubleType(), True),
    StructField("F1 Score", DoubleType(), True),
])
df = spark.createDataFrame([], schema)


In [0]:
#loop for classes 0.0 to 2.0 for 3 classes of data labels
for i in range(3):
    tempList = [str(float(i)),metrics.truePositiveRate(float(i)),metrics.falsePositiveRate(float(i)),metrics.precision(float(i)),metrics.recall(float(i)),metrics.fMeasure(float(i))] 
    tempDf = spark.createDataFrame([tempList])
    df = df.union(tempDf)

display(df)

Label,True Positive Rate,False Positive Rate,Precision,Recall,F1 Score
0.0,0.9155701754385965,0.5187265917602997,0.7508992805755396,0.9155701754385965,0.8250988142292491
1.0,0.36303630363036304,0.07174103237095363,0.5729166666666666,0.36303630363036304,0.4444444444444444
2.0,0.3852813852813853,0.043621399176954734,0.6267605633802817,0.3852813852813853,0.47721179624664883


In [0]:
print(metrics.accuracy)

0.715076071922545


In [0]:
# Model Accuracy - 71.5%

In [0]:
Output file link - "/Data_repository/main/Assig_Q2_output.csv"
Metrics File link - "/Data_repository/main/Assig_Q2_metrics.csv"